In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import dataset_utils
import NN

In [2]:
import dataset_utils

In [3]:
def flatten_and_pad(point_cloud, target_length):
    # Flatten the point cloud into a 2D array with X, Y, Z coordinates
    # Ensure the data is padded or truncated to match the target length
    num_points = len(point_cloud)
    padded_point_cloud = np.zeros((target_length, 3))
    padded_point_cloud[:num_points] = point_cloud
    return padded_point_cloud

In [6]:
DATA_DIR = "pointnet_data"
NUM_POINTS = 40
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: sit
processing class: stand


In [7]:
print(train_points)

[[[-1.992e+00  9.404e-01  1.846e+00  0.000e+00  1.770e+02]
  [-1.729e-01  1.561e+00  9.053e-01  3.757e-01  2.860e+02]
  [-2.152e+00  2.051e+00  8.691e-01  0.000e+00  2.190e+02]
  ...
  [-3.323e-01  1.723e+00  8.232e-01  2.505e-01  2.740e+02]
  [-1.992e+00  9.424e-01  1.860e+00  0.000e+00  1.740e+02]
  [-4.316e-01  1.580e+00  8.936e-01  2.505e-01  3.350e+02]]

 [[ 4.678e-01  1.580e-01  1.717e+00  0.000e+00  1.350e+02]
  [-2.476e-01  1.596e+00  9.971e-01  2.505e-01  2.850e+02]
  [-2.983e-01  1.602e+00  9.014e-01  5.010e-01  2.470e+02]
  ...
  [ 4.678e-01  1.816e-01  1.730e+00  0.000e+00  1.370e+02]
  [-2.764e-01  1.642e+00  8.115e-01 -1.252e-01  1.740e+02]
  [-2.025e+00  2.971e-01  1.512e+00  0.000e+00  1.380e+02]]

 [[ 6.455e-01  3.867e-01 -8.789e-02  0.000e+00  1.180e+02]
  [ 9.551e-01  3.816e+00  8.389e-01 -1.252e-01  2.110e+02]
  [-5.078e-02  2.221e+00  8.594e-02  2.505e-01  1.940e+02]
  ...
  [ 7.998e-01  3.855e+00  9.609e-01 -1.252e-01  3.150e+02]
  [ 8.462e-01  3.906e+00  9.795e-0

In [8]:
from tensorflow import data as tf_data
import keras
import PointNET

keras.utils.set_random_seed(seed=42)

In [9]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [10]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [11]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [12]:
def model_structure_1D(input_shape, output_shape):
    _model_input = Input(shape=input_shape)
    _model = Conv1D(32, kernel_size=1, padding='same', activation='relu')(_model_input)
    _model = MaxPooling1D(pool_size=2)(_model)
    _model = Flatten()(_model)
    _model = Dense(output_shape, activation='softmax')(_model)
    model = Model(inputs=_model_input, outputs=_model)
    return model


In [13]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [14]:
input_shape = (40, 5)  # Example input shape for 3D data
output_shape = 3  # Number of classes or output units
model = model_structure_1D(input_shape, output_shape)

AttributeError: 'tuple' object has no attribute 'rank'

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))

# 2. LSTM layer
model.add(layers.LSTM(50, return_sequences=True))
model.add(layers.LSTM(50))

# 3. Dense layers
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(3, activation='linear'))  # Output layer (you can adjust the output size and activation)


In [16]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, None, 64)       │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, None, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, None, 50)       │        23,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,825 (182.91 KB)

 Trainable params: 46,825 (182.91 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# training

history = model.fit(train_points, train_labels, epochs=100, batch_size=32, validation_data=(test_points, test_labels), verbose=4)
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=2)

pickle.dump(model, open("model.pkl", "wb"))
